### **Importación de datos, librerías y definición de funciones**

In [1]:
import pandas as pd
import json

In [2]:
# data = pd.read_csv("../data/train.csv")

# df = data

# para v2 cargar --> convertir todo esto en un for donde se van limpiando los datos segun llegan (las columnas ya se añaden limpias al dataframe final)
misc = pd.read_csv("../data/segregated/misc_v2.csv")
device = pd.read_csv("../data/segregated/device_v2.csv")
geoNetwork = pd.read_csv("../data/segregated/geoNetwork_v2.csv")
totals = pd.read_csv("../data/segregated/totals_v2.csv")
trafficSource = pd.read_csv("../data/segregated/trafficSource_v2.csv")
# hitsFiles = glob.glob("*_hists.csv")
# for fl in hitsFiles:
  # hits = pd.read_csv("data/fl.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# device = pd.read_csv("../data/segregated/device_v2_new.csv")
# device = device.sample(903653)

In [4]:
# device.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
# device

In [5]:
print(misc.shape)
print(device.shape)
print(geoNetwork.shape)
print(totals.shape)
print(trafficSource.shape)

(1799982, 9)
(1799982, 4)
(1799982, 4)
(1799982, 4)
(1799982, 4)


In [6]:
misc.drop(columns=['Unnamed: 0', 'visitId'], axis=1, inplace=True)
device.drop(columns=['Unnamed: 0', 'fullVisitorId', 'visitId'], axis=1, inplace=True)
geoNetwork.drop(columns=['Unnamed: 0', 'fullVisitorId', 'visitId'], axis=1, inplace=True)
totals.drop(columns=['Unnamed: 0', 'fullVisitorId', 'visitId'], axis=1, inplace=True)
trafficSource.drop(columns=['Unnamed: 0', 'fullVisitorId', 'visitId'], axis=1, inplace=True)

In [7]:
# recursive flatten function from https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10
# Call example:
  # flattenCol = [flatten_json(json.loads(d)) for d in df['trafficSource']]
  # dataFrameOfFlattenCol = pd.DataFrame(flattenCol)
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

  
# Call example:
  # df = extract_day(df, 'month')
def extract_date_info(df, parametro):
    if parametro == "date":
        df["date"] = [d.date() for d in df["visitStartTime"]]
    if parametro == "time":
        df["time"] = [d.time() for d in df["visitStartTime"]]
    if parametro == "day":
        df["day"] = df["visitStartTime"].dt.day
    if parametro == "month":
        df['month'] = df["visitStartTime"].dt.month
    if parametro == "weekday":
        df['weekday'] = df["visitStartTime"].dt.weekday
    if parametro == "weekofyear": 
        df['weekofyear'] = df["visitStartTime"].dt.weekofyear
    return df

def catch(func, handle=lambda e : e, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return handle(e)

### Limpieza 


In [8]:
# De misc mantenemos:
  # fullVisitorId
  # visitStartTime
  # channelGrouping

misc["visitStartTime"] = pd.to_datetime(misc["visitStartTime"], unit='s')
misc.drop(['date', 'socialEngagementType'], axis=1, inplace=True)

df = misc.copy()

In [ ]:
# pd.set_option('display.max_colwidth', 1000)
# device.iloc[0]

In [11]:
# De device mantenemos:
  # browser
  # deviceCategory
  # operatingSystem

numberExc = 0

deviceFlatten = [flatten_json(catch(lambda: json.loads(d))) for d in device['device']]

devi = pd.DataFrame(deviceFlatten)

# quest: language?
devi.drop(columns=['', 'browserSize', 'browserVersion', 'flashVersion', 'language', 
    'mobileDeviceBranding', 'mobileDeviceInfo', 'mobileDeviceMarketingName',
    'mobileDeviceModel', 'mobileInputSelector', 'operatingSystemVersion',
    'screenColors','screenResolution'], inplace=True)
isMobile = {False : 0, True : 1}
browser = {'Chrome': 1, 'Safari': 2, 'Firefox': 3, 'Internet Explorer': 4, 'Edge': 5, 
    'Android Webview': 0, 'Safari (in-app)': 0, 'Opera Mini': 0, 'Opera': 0,
    'UC Browser': 0}
category = {'desktop': 1, 'mobile': 2, 'tablet': 3}
operatingSystem = {'Windows': 1, 'Macintosh': 2, 'Android': 3, 'iOS': 4,'Linux': 5,
    'Chrome OS': 6, '(not set)': 0, 'Windows Phone': 0, 'BlackBerry': 0, 'Samsung': 0}

#we map our data to its corresponding value
devi["isMobile"] = devi["isMobile"].map(isMobile)
devi["browser"] = devi["browser"].map(browser)
devi["deviceCategory"] = devi["deviceCategory"].map(category)
devi["operatingSystem"] = devi["operatingSystem"].map(operatingSystem)

devi.drop(columns=['isMobile'], inplace=True)
df = df.join(devi)

In [12]:
# print ("There we", numberExc, "exceptions")

In [13]:
devi.shape

(1799982, 3)

In [14]:
df.shape

(1799982, 8)

In [15]:
device['device'].shape

(1799982,)

In [17]:
# De geoNetwork mantenemos:
  # coutry
  # region (que contiene ciudad si no está disponible)
  # city

geoNetworkFlatten = [flatten_json(catch(lambda: json.loads(d))) for d in geoNetwork['geoNetwork']]
geo = pd.DataFrame(geoNetworkFlatten)
geo.drop(['latitude','longitude','networkLocation','cityId','continent','metro'], axis=1, inplace=True)

# QUEST: check that it works in the same way
# geo['region'].apply(lambda x: geo['country'] if x == '(not set)' else x)
# geo['region'].apply(lambda x: geo['country'] if x == 'not available in demo dataset' else x)


def region_country(geo):
    if geo['region'] == '(not set)':
        return geo['country']
    elif geo['region'] == 'not available in demo dataset':
        return geo['country']
    else:
        return geo['region']

geo['region']=geo.apply(region_country,axis=1)

# df.drop(['geoNetwork'], axis=1, inplace=True)
df = df.join(geo)
# df = pd.concat([df, geo])

In [21]:
# De totals mantenemos:
  # hits
  # transactionRevenue
  # newVisits

#Transform json
totalsList = [flatten_json(catch(lambda: json.loads(d))) for d in totals['totals']]
totals_df = pd.DataFrame(totalsList)
# totals_df = totals['totals'].apply(catch(lambda: json.loads)).apply(pd.Series)

#Remove visits column as it does not provide any information
totals_df = totals_df.drop(['', 'visits'], axis=1)

#Change nan in "newVisits" for 0. Binary, 1 it's new visit, 0 it's not. 
totals_df['newVisits'] = totals_df['newVisits'].fillna(0)

#Change nan in "transactionRevenue" for 0. 
totals_df['transactionRevenue'] = totals_df['transactionRevenue'].fillna(0)

#Change nan in "bounces" for 0
totals_df['bounces'] = totals_df['bounces'].fillna(0)

#There are some nan in pageviews but it is not trivial to find a value we can change these nan by logically.
#The corrlation with hits is 0.984, so we can keep hits that does not have any nan value. Keeping both could be redundant.
totals_df = totals_df.drop(['pageviews'], axis=1)

#Not 100% proven yet, but i have a big feeling that bounces is mainly 1 when there is only 1 hit by defenition.
#This means that even though the correlation doesn't provide a big value beetween them two, bounces is not giving much 
#additional info. Meaning using it is not relevant.
totals_df = totals_df.drop(['bounces'], axis=1)

# df.drop(['totals'], axis=1, inplace=True)
df = df.join(totals_df)
# df = pd.concat([df, totals_df])

In [23]:
# De trafficSource mantenemos:
  # adHasContent
  # adPosition
  # campaign
  # isTrueDirect
  # medium
  # source

trafficFlatten = [flatten_json(catch(lambda: json.loads(d))) for d in trafficSource['trafficSource']]
traffic = pd.DataFrame(trafficFlatten)
traffic.drop(['', 'adwordsClickInfo_criteriaParameters', 'adwordsClickInfo_adNetworkType',
    'adwordsClickInfo_gclId', 'adwordsClickInfo_isVideoAd', 'adwordsClickInfo_page',
    'keyword', 'referralPath', ], axis=1, inplace=True)

# TO DO check campaignCode

# adContent to bool
traffic['adContent'] = traffic['adContent'].map(lambda x: 0 if str(x) == 'NoneType' else 1)

# rename to a simpler name
traffic.rename(index=str, columns={"adwordsClickInfo_page": "adPosition", "adContent": "adwordsClickInfo_slot", "": "adPosition"})

# TO DO: traffic['source'] regex for the main domain (last whatevs.com), and drop less than 1000


# TO DO: traffc['medium']
#     merge (not set) and (none)
#     cpc -> Cost Per Click
#     affiliate -> Affiliate marketing is a type of performance-based marketing in which a business rewards one or more affiliates for each visitor or customer brought by the affiliate's own marketing efforts.
#     cpm -> Cost per impression
# entender qué implican estos valores. ¿cpc vienen de search y cpm de webs? ¿Se juntan? ¿Importan?


# df.drop(['trafficSource'], axis=1, inplace=True)
df = df.join(traffic)
# df = pd.concat([df, traffic])

In [24]:
# hits (only in v2)

In [28]:
list(df.columns.values)

['Unnamed: 1',
 'channelGrouping',
 'fullVisitorId',
 'visitNumber',
 'visitStartTime',
 'browser',
 'deviceCategory',
 'operatingSystem',
 '',
 'city',
 'country',
 'networkDomain',
 'region',
 'subContinent',
 'hits',
 'newVisits',
 'sessionQualityDim',
 'timeOnSite',
 'totalTransactionRevenue',
 'transactionRevenue',
 'transactions',
 'adContent',
 'adwordsClickInfo_slot',
 'campaign',
 'campaignCode',
 'isTrueDirect',
 'medium',
 'source']

In [30]:
df.drop(columns=["", "Unnamed: 1"], axis=1, inplace=True)

In [31]:
df.shape

(1799982, 26)

In [32]:
df.head()

,channelGrouping,fullVisitorId,visitNumber,visitStartTime,browser,deviceCategory,operatingSystem,city,country,networkDomain,...,totalTransactionRevenue,transactionRevenue,transactions,adContent,adwordsClickInfo_slot,campaign,campaignCode,isTrueDirect,medium,source
0,Organic Search,538928163114544921,1.0,2017-05-29 16:07:23,1.0,3.0,3.0,not available in demo dataset,United States,rr.com,...,NaN,0,NaN,1,NaN,(not set),NaN,NaN,organic,google
1,Organic Search,5863936343673668667,2.0,2017-05-29 19:43:31,1.0,1.0,1.0,Bengaluru,India,122-airtelbroadband.in,...,NaN,0,NaN,1,NaN,(not set),NaN,True,organic,google
2,Organic Search,9993197829693068679,9.0,2017-05-29 23:09:51,2.0,2.0,4.0,San Jose,United States,comcast.net,...,NaN,0,NaN,1,NaN,(not set),NaN,True,organic,google
3,Referral,635666386021193101,13.0,2017-05-29 09:20:43,1.0,1.0,1.0,Osaka,Japan,i-revonet.jp,...,NaN,0,NaN,1,NaN,(not set),NaN,True,referral,analytics.google.com
4,Organic Search,7772696018151280169,1.0,2017-05-29 10:19:06,1.0,1.0,1.0,not available in demo dataset,Spain,jazztel.es,...,NaN,0,NaN,1,NaN,(not set),NaN,NaN,organic,google


In [27]:
df.to_csv("../data/train_v2_cleaned.csv")
df.to_pickle("../data/train_v2_cleaned.csv")

KeyboardInterrupt: 

### **Transformación**

### ** Minería de datos **

In [ ]:
traffic

### ** Interpretación y evaluación **

In [ ]:
import numpy as np

In [ ]:
mixed = pd.DataFrame(np.random.randn(6,4),columns=list('ABCD'))
mixed['A'].iloc[0] = 'A'

In [ ]:
mixed['B'].dtype